# 스탠스 분석 모델 학습 (개선 버전)

KoBERT 기반 뉴스 기사 스탠스 분류 모델 학습
- **3-class 분류**: 옹호(0), 중립(1), 비판(2)
- **개선사항**: Early Stopping, Weight Decay, Gradient Clipping, LR Scheduling

---

## 1️⃣ 환경 설정

In [ ]:
# Google Colab 환경 확인
try:
    import google.colab
    IN_COLAB = True
    print("✅ Google Colab 환경")
except:
    IN_COLAB = False
    print("✅ 로컬 환경")

In [ ]:
# 필수 패키지 설치
!pip install torch transformers scikit-learn pandas numpy matplotlib seaborn tqdm sentencepiece -q

In [ ]:
# 라이브러리 임포트
import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader
from torch.optim import AdamW
from torch.optim.lr_scheduler import ReduceLROnPlateau

from transformers import BertModel, AutoTokenizer

import pandas as pd
import numpy as np
import random

from sklearn.metrics import classification_report, confusion_matrix, accuracy_score
from sklearn.model_selection import train_test_split
from sklearn.utils.class_weight import compute_class_weight

import matplotlib.pyplot as plt
import seaborn as sns
from tqdm.auto import tqdm

import warnings
warnings.filterwarnings('ignore')

print("✅ 라이브러리 임포트 완료")
print(f"PyTorch 버전: {torch.__version__}")
print(f"CUDA 사용 가능: {torch.cuda.is_available()}")

In [ ]:
# 디바이스 설정
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f"사용 디바이스: {device}")

if torch.cuda.is_available():
    print(f"GPU: {torch.cuda.get_device_name(0)}")
    print(f"GPU 메모리: {torch.cuda.get_device_properties(0).total_memory / 1e9:.2f} GB")

## 2️⃣ 하이퍼파라미터 설정 (개선 버전)

In [ ]:
# ==========================================
# 개선된 하이퍼파라미터 (과적합 방지 강화!)
# ==========================================

# 모델 파라미터
MAX_LENGTH = 128        # 최대 시퀀스 길이
DROPOUT = 0.4           # 0.3 → 0.4로 증가 (더 강한 regularization)

# 학습 파라미터
BATCH_SIZE = 16         # 배치 크기
EPOCHS = 15             # 최대 epoch (early stopping으로 조기 중단)
LEARNING_RATE = 2e-5    # 5e-5 → 2e-5로 낮춤 (더 안정적인 학습)
WEIGHT_DECAY = 0.01     # Weight decay 추가! (L2 regularization)
MAX_GRAD_NORM = 1.0     # Gradient clipping (exploding gradient 방지)

# Early Stopping 파라미터
EARLY_STOP_PATIENCE = 3 # 3 epoch 동안 개선 없으면 중단
EARLY_STOP_MIN_DELTA = 0.001  # 개선으로 간주할 최소 변화량

# 데이터 분할 (샘플 데이터 사용 시)
TEST_SIZE = 0.1
VAL_SIZE = 0.1

# 시드 설정
SEED = 42
random.seed(SEED)
np.random.seed(SEED)
torch.manual_seed(SEED)
torch.cuda.manual_seed_all(SEED)

print("✅ 개선된 하이퍼파라미터 설정 완료!")
print(f"\n📊 주요 변경사항:")
print(f"   - DROPOUT: 0.3 → {DROPOUT}")
print(f"   - LEARNING_RATE: 5e-5 → {LEARNING_RATE}")
print(f"   - WEIGHT_DECAY: 0 → {WEIGHT_DECAY} (새로 추가!)")
print(f"   - MAX_GRAD_NORM: {MAX_GRAD_NORM} (새로 추가!)")
print(f"   - EARLY_STOP_PATIENCE: {EARLY_STOP_PATIENCE}")

## 3️⃣ 데이터 로드

In [ ]:
# 데이터 로드 방식 선택
USE_REAL_DATA = True  # True: 실제 CSV 파일 사용, False: 샘플 데이터 사용

if USE_REAL_DATA:
    if IN_COLAB:
        # Colab: 파일 업로드
        from google.colab import files
        print("📤 데이터 파일을 업로드하세요 (train_dataset.csv, val_dataset.csv, test_dataset.csv)")
        uploaded = files.upload()
        
        train_df = pd.read_csv('train_dataset.csv')
        val_df = pd.read_csv('val_dataset.csv')
        test_df = pd.read_csv('test_dataset.csv')
    else:
        # 로컬: 파일 경로 지정
        train_df = pd.read_csv('data/train_dataset.csv')
        val_df = pd.read_csv('data/val_dataset.csv')
        test_df = pd.read_csv('data/test_dataset.csv')
    
    print(f"✅ 데이터 로드 완료")
    print(f"   - Train: {len(train_df)}개")
    print(f"   - Val: {len(val_df)}개")
    print(f"   - Test: {len(test_df)}개")
    
else:
    # 샘플 데이터 생성
    sample_data = {
        'content': [
            # 옹호 (0)
            "정부의 새로운 정책은 경제 활성화에 크게 기여할 것으로 기대됩니다.",
            "이번 개혁안은 국민들의 삶의 질 향상에 도움이 될 것입니다.",
            "대통령의 결정은 국가 발전을 위한 올바른 방향입니다.",
            "정부의 적극적인 대응이 위기 극복에 효과적이었습니다.",
            "새로운 법안은 사회 정의 실현에 중요한 역할을 할 것입니다.",
            "정책의 긍정적 효과가 곳곳에서 나타나고 있습니다.",
            "정부의 노력으로 많은 문제가 해결되고 있습니다.",
            "이번 조치는 국민을 위한 현명한 선택이었습니다.",
            # 중립 (1)
            "정부가 새로운 정책을 발표했습니다.",
            "오늘 국회에서 법안이 통과되었습니다.",
            "대통령이 기자회견을 열었습니다.",
            "정부는 새로운 예산안을 제출했습니다.",
            "관련 부처가 대책을 논의 중입니다.",
            "전문가들은 다양한 의견을 제시했습니다.",
            "정책 시행 일정이 발표되었습니다.",
            "여야가 협상을 진행하고 있습니다.",
            # 비판 (2)
            "정부의 정책은 현실을 반영하지 못하고 있습니다.",
            "이번 결정은 국민의 뜻과 거리가 멉니다.",
            "정책의 부작용이 우려됩니다.",
            "정부의 대응이 미흡하다는 비판이 제기되고 있습니다.",
            "전문가들은 이번 조치의 효과에 의문을 표했습니다.",
            "정책 실행 과정에서 많은 문제점이 드러났습니다.",
            "국민들의 불만이 높아지고 있습니다.",
            "이번 법안은 재검토가 필요해 보입니다."
        ],
        'label': [0]*8 + [1]*8 + [2]*8
    }
    
    df = pd.DataFrame(sample_data)
    
    # 데이터 분할
    train_df, temp_df = train_test_split(df, test_size=0.2, random_state=SEED, stratify=df['label'])
    val_df, test_df = train_test_split(temp_df, test_size=0.5, random_state=SEED, stratify=temp_df['label'])
    
    print(f"✅ 샘플 데이터 생성 완료")
    print(f"   - Train: {len(train_df)}개")
    print(f"   - Val: {len(val_df)}개")
    print(f"   - Test: {len(test_df)}개")

In [ ]:
# 데이터 확인
print("\n📊 Train 데이터 샘플:")
print(train_df.head())

print("\n📊 클래스 분포 (Train):")
label_counts = train_df['label'].value_counts().sort_index()
label_names = ['옹호', '중립', '비판']
for label, count in label_counts.items():
    print(f"   {label} ({label_names[label]}): {count}개 ({count/len(train_df)*100:.1f}%)")

## 4️⃣ 데이터 전처리

In [ ]:
# KoBERT 토크나이저 로드
# monologg/kobert 사용 (더 안정적인 토크나이징)
MODEL_NAME = 'monologg/kobert'

try:
    tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME, trust_remote_code=True)
    print(f"✅ 토크나이저 로드 완료: {MODEL_NAME}")
    print(f"   Vocab size: {len(tokenizer)}")
    
    # 토크나이저 테스트
    test_text = "정부의 새로운 정책이 발표되었습니다."
    test_encoded = tokenizer(test_text, return_tensors='pt')
    test_decoded = tokenizer.decode(test_encoded['input_ids'][0], skip_special_tokens=False)
    
    print(f"\n✅ 토크나이저 동작 확인:")
    print(f"   원본: {test_text}")
    print(f"   디코딩: {test_decoded}")
    
    # [UNK] 토큰이 많으면 경고
    if test_decoded.count('[UNK]') > len(test_text) * 0.3:
        print("⚠️  경고: [UNK] 토큰이 많습니다. 다른 모델을 시도하세요.")
    else:
        print("✅ 정상 작동 중")
        
except Exception as e:
    print(f"❌ 토크나이저 로드 실패: {e}")
    print("\n대체 방법 시도: skt/kobert-base-v1")
    MODEL_NAME = 'skt/kobert-base-v1'
    tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME)
    print(f"✅ 토크나이저 로드 완료: {MODEL_NAME}")

In [ ]:
# 토크나이저 상세 테스트 (선택 사항)
# 실제 뉴스 텍스트로 토크나이저 동작 확인

test_news = """전북도의회는 30일 입장문을 내고 "정부가 확정한 제4차 국가철도망 구축계획은 광역권 없는 전북을 철저하게 외면한 결과"라며 강력하게 항의했다."""

print("="*60)
print("토크나이저 상세 테스트")
print("="*60)
print(f"\n원본 텍스트:\n{test_news}\n")

# 토크나이징
encoded = tokenizer(
    test_news,
    add_special_tokens=True,
    max_length=128,
    padding='max_length',
    truncation=True,
    return_tensors='pt'
)

print(f"Input IDs 길이: {len(encoded['input_ids'][0])}")
print(f"Attention Mask 길이: {len(encoded['attention_mask'][0])}")

# 디코딩
decoded_with_special = tokenizer.decode(encoded['input_ids'][0], skip_special_tokens=False)
decoded_without_special = tokenizer.decode(encoded['input_ids'][0], skip_special_tokens=True)

print(f"\n디코딩 결과 (스페셜 토큰 포함):\n{decoded_with_special[:200]}...")
print(f"\n디코딩 결과 (스페셜 토큰 제외):\n{decoded_without_special[:200]}...")

# [UNK] 비율 체크
unk_count = decoded_with_special.count('[UNK]')
unk_ratio = unk_count / len(test_news) if len(test_news) > 0 else 0

print(f"\n[UNK] 토큰 개수: {unk_count}")
print(f"[UNK] 비율: {unk_ratio*100:.2f}%")

if unk_ratio > 0.3:
    print("\n⚠️  경고: [UNK] 토큰 비율이 높습니다 (30% 이상)")
    print("   → 토크나이저가 제대로 작동하지 않을 수 있습니다.")
    print("   → 다른 모델을 시도하거나 설치를 확인하세요.")
elif unk_ratio > 0.1:
    print("\n⚠️  주의: [UNK] 토큰 비율이 다소 높습니다 (10% 이상)")
else:
    print("\n✅ 토크나이저 정상 작동!")
    print("   → 다음 단계로 진행하세요.")

print("="*60)

In [ ]:
# Dataset 클래스 정의
class StanceDataset(Dataset):
    """
    스탠스 분류를 위한 Dataset 클래스
    """
    def __init__(self, texts, labels, tokenizer, max_length):
        self.texts = texts
        self.labels = labels
        self.tokenizer = tokenizer
        self.max_length = max_length
    
    def __len__(self):
        return len(self.texts)
    
    def __getitem__(self, idx):
        text = str(self.texts[idx])
        label = self.labels[idx]
        
        encoding = self.tokenizer(
            text,
            add_special_tokens=True,
            max_length=self.max_length,
            padding='max_length',
            truncation=True,
            return_tensors='pt'
        )
        
        return {
            'input_ids': encoding['input_ids'].squeeze(0),
            'attention_mask': encoding['attention_mask'].squeeze(0),
            'label': torch.tensor(label, dtype=torch.long)
        }

print("✅ StanceDataset 클래스 정의 완료")

In [ ]:
# DataLoader 생성
train_texts = train_df['content'].tolist()
train_labels = train_df['label'].tolist()

val_texts = val_df['content'].tolist()
val_labels = val_df['label'].tolist()

test_texts = test_df['content'].tolist()
test_labels = test_df['label'].tolist()

train_dataset = StanceDataset(train_texts, train_labels, tokenizer, MAX_LENGTH)
val_dataset = StanceDataset(val_texts, val_labels, tokenizer, MAX_LENGTH)
test_dataset = StanceDataset(test_texts, test_labels, tokenizer, MAX_LENGTH)

train_loader = DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=BATCH_SIZE, shuffle=False)
test_loader = DataLoader(test_dataset, batch_size=BATCH_SIZE, shuffle=False)

print("✅ DataLoader 생성 완료")
print(f"   - Train batches: {len(train_loader)}")
print(f"   - Val batches: {len(val_loader)}")
print(f"   - Test batches: {len(test_loader)}")

## 5️⃣ 모델 정의

In [ ]:
# 스탠스 분류 모델
class StanceClassifier(nn.Module):
    """
    KoBERT 기반 스탠스 분류 모델
    """
    def __init__(self, model_name, num_classes=3, dropout=0.4):
        super(StanceClassifier, self).__init__()
        self.bert = BertModel.from_pretrained(model_name)
        self.dropout = nn.Dropout(dropout)
        self.classifier = nn.Linear(self.bert.config.hidden_size, num_classes)
    
    def forward(self, input_ids, attention_mask):
        outputs = self.bert(
            input_ids=input_ids,
            attention_mask=attention_mask
        )
        
        # [CLS] 토큰의 hidden state 사용
        pooled_output = outputs.pooler_output
        pooled_output = self.dropout(pooled_output)
        logits = self.classifier(pooled_output)
        
        return logits

print("✅ StanceClassifier 클래스 정의 완료")

In [ ]:
# 모델 초기화
model = StanceClassifier(
    model_name=MODEL_NAME,
    num_classes=3,
    dropout=DROPOUT
).to(device)

# 모델 파라미터 수 출력
total_params = sum(p.numel() for p in model.parameters())
trainable_params = sum(p.numel() for p in model.parameters() if p.requires_grad)

print(f"✅ 모델 초기화 완료")
print(f"   - 총 파라미터: {total_params:,}")
print(f"   - 학습 가능 파라미터: {trainable_params:,}")
print(f"   - Dropout: {DROPOUT}")

## 6️⃣ Early Stopping 클래스

In [ ]:
# Early Stopping 구현
class EarlyStopping:
    """
    Early Stopping: 검증 손실이 개선되지 않으면 학습 중단
    """
    def __init__(self, patience=3, min_delta=0.001, verbose=True):
        """
        Args:
            patience: 개선이 없어도 기다릴 epoch 수
            min_delta: 개선으로 간주할 최소 변화량
            verbose: 로그 출력 여부
        """
        self.patience = patience
        self.min_delta = min_delta
        self.verbose = verbose
        
        self.counter = 0
        self.best_loss = None
        self.early_stop = False
        self.best_model_state = None
    
    def __call__(self, val_loss, model):
        """
        검증 손실을 체크하고 early stop 여부 결정
        """
        if self.best_loss is None:
            self.best_loss = val_loss
            self.save_checkpoint(val_loss, model)
        
        elif val_loss > self.best_loss - self.min_delta:
            self.counter += 1
            if self.verbose:
                print(f'⚠️  EarlyStopping counter: {self.counter}/{self.patience}')
                print(f'   현재 Val Loss: {val_loss:.4f}, 최고 Val Loss: {self.best_loss:.4f}')
            
            if self.counter >= self.patience:
                self.early_stop = True
                if self.verbose:
                    print(f'\n🛑 Early Stopping! {self.patience} epoch 동안 개선 없음')
        
        else:
            self.save_checkpoint(val_loss, model)
            self.counter = 0
    
    def save_checkpoint(self, val_loss, model):
        """최고 성능 모델 저장"""
        if self.verbose:
            if self.best_loss is not None:
                print(f'✅ Val Loss 개선: {self.best_loss:.4f} → {val_loss:.4f}')
            else:
                print(f'✅ 최초 모델 저장 (Val Loss: {val_loss:.4f})')
        
        self.best_loss = val_loss
        # 모델 state dict 저장 (메모리에)
        self.best_model_state = {
            key: value.cpu().clone() for key, value in model.state_dict().items()
        }
    
    def load_best_model(self, model):
        """최고 성능 모델 로드"""
        if self.best_model_state is not None:
            model.load_state_dict(
                {key: value.to(model.bert.device)
                 for key, value in self.best_model_state.items()}
            )
            if self.verbose:
                print(f'✅ 최고 성능 모델 로드 완료 (Val Loss: {self.best_loss:.4f})')

print("✅ EarlyStopping 클래스 정의 완료!")

## 7️⃣ Optimizer 및 Scheduler 설정

In [ ]:
# ==========================================
# Optimizer 및 Scheduler (개선!)
# ==========================================

# 클래스 가중치 계산
class_weights = compute_class_weight(
    'balanced',
    classes=np.unique(train_labels),
    y=train_labels
)
class_weights = torch.FloatTensor(class_weights).to(device)

print("\n클래스별 가중치:")
for i, w in enumerate(class_weights):
    label_name = ['옹호', '중립', '비판'][i]
    print(f"  {i} ({label_name}): {w:.4f}")

# Loss Function (클래스 가중치 적용)
criterion = nn.CrossEntropyLoss(weight=class_weights)

# Optimizer (Weight Decay 추가!)
optimizer = AdamW(
    model.parameters(),
    lr=LEARNING_RATE,
    weight_decay=WEIGHT_DECAY,  # L2 regularization!
    eps=1e-8
)

# Scheduler (ReduceLROnPlateau)
scheduler = ReduceLROnPlateau(
    optimizer,
    mode='min',        # validation loss를 최소화
    factor=0.5,        # LR을 절반으로
    patience=2,        # 2 epoch 동안 개선 없으면 LR 감소
    verbose=True,      # 로그 출력
    min_lr=1e-7        # 최소 learning rate
)

# Early Stopping 초기화
early_stopping = EarlyStopping(
    patience=EARLY_STOP_PATIENCE,
    min_delta=EARLY_STOP_MIN_DELTA,
    verbose=True
)

# 학습 기록 저장용
history = {
    'train_loss': [],
    'train_acc': [],
    'val_loss': [],
    'val_acc': [],
    'learning_rates': []  # LR 변화 추적
}

print("\n✅ Optimizer, Scheduler, Early Stopping 설정 완료!")
print(f"   Weight Decay (L2): {WEIGHT_DECAY}")
print(f"   Initial LR: {LEARNING_RATE}")
print(f"   Early Stop Patience: {EARLY_STOP_PATIENCE}")

## 8️⃣ 학습 및 검증 함수

In [ ]:
# ==========================================
# 학습 함수 (Gradient Clipping 추가!)
# ==========================================

def train_epoch(model, data_loader, criterion, optimizer, device, max_grad_norm=1.0):
    """
    1 epoch 학습
    
    Args:
        max_grad_norm: gradient clipping 임계값
    """
    model.train()
    total_loss = 0
    correct = 0
    total = 0
    
    progress_bar = tqdm(data_loader, desc='Training')
    
    for batch in progress_bar:
        input_ids = batch['input_ids'].to(device)
        attention_mask = batch['attention_mask'].to(device)
        labels = batch['label'].to(device)
        
        optimizer.zero_grad()
        
        # Forward
        outputs = model(input_ids, attention_mask)
        loss = criterion(outputs, labels)
        
        # Backward
        loss.backward()
        
        # Gradient Clipping (추가!)
        torch.nn.utils.clip_grad_norm_(model.parameters(), max_grad_norm)
        
        optimizer.step()
        
        # 통계
        total_loss += loss.item()
        _, predicted = torch.max(outputs, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()
        
        # Progress bar 업데이트
        progress_bar.set_postfix({
            'loss': f'{loss.item():.4f}',
            'acc': f'{100 * correct / total:.2f}%'
        })
    
    avg_loss = total_loss / len(data_loader)
    accuracy = correct / total
    
    return avg_loss, accuracy


def eval_model(model, data_loader, criterion, device):
    """모델 평가 (검증/테스트)"""
    model.eval()
    total_loss = 0
    correct = 0
    total = 0
    
    with torch.no_grad():
        progress_bar = tqdm(data_loader, desc='Evaluating')
        
        for batch in progress_bar:
            input_ids = batch['input_ids'].to(device)
            attention_mask = batch['attention_mask'].to(device)
            labels = batch['label'].to(device)
            
            outputs = model(input_ids, attention_mask)
            loss = criterion(outputs, labels)
            
            total_loss += loss.item()
            _, predicted = torch.max(outputs, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()
            
            progress_bar.set_postfix({
                'loss': f'{loss.item():.4f}',
                'acc': f'{100 * correct / total:.2f}%'
            })
    
    avg_loss = total_loss / len(data_loader)
    accuracy = correct / total
    
    return avg_loss, accuracy

print("✅ 개선된 학습/검증 함수 정의 완료!")
print("   - Gradient Clipping 추가")

## 9️⃣ 학습 실행

In [ ]:
# ==========================================
# 학습 루프 (Early Stopping 적용!)
# ==========================================

print("\n" + "="*60)
print("학습 시작 (Early Stopping 활성화)")
print("="*60)
print(f"최대 Epoch: {EPOCHS}")
print(f"Early Stop Patience: {EARLY_STOP_PATIENCE}")
print(f"Batch Size: {BATCH_SIZE}")
print(f"Learning Rate: {LEARNING_RATE}")
print(f"Weight Decay: {WEIGHT_DECAY}")
print(f"Dropout: {DROPOUT}")
print("="*60 + "\n")

best_val_acc = 0.0
best_epoch = 0

for epoch in range(EPOCHS):
    print(f"\n{'='*60}")
    print(f"Epoch {epoch + 1}/{EPOCHS}")
    print(f"{'='*60}")
    
    # 현재 Learning Rate 출력
    current_lr = optimizer.param_groups[0]['lr']
    print(f"📊 Current Learning Rate: {current_lr:.2e}")
    
    # 학습
    train_loss, train_acc = train_epoch(
        model, train_loader, criterion, optimizer, device,
        max_grad_norm=MAX_GRAD_NORM
    )
    print(f"\n📊 Train Loss: {train_loss:.4f}, Train Acc: {train_acc*100:.2f}%")
    
    # 검증
    val_loss, val_acc = eval_model(model, val_loader, criterion, device)
    print(f"📊 Val Loss: {val_loss:.4f}, Val Acc: {val_acc*100:.2f}%")
    
    # 학습률 스케줄러 업데이트
    scheduler.step(val_loss)
    
    # 기록 저장
    history['train_loss'].append(train_loss)
    history['train_acc'].append(train_acc)
    history['val_loss'].append(val_loss)
    history['val_acc'].append(val_acc)
    history['learning_rates'].append(current_lr)
    
    # 최고 성능 추적
    if val_acc > best_val_acc:
        best_val_acc = val_acc
        best_epoch = epoch + 1
        print(f"✨ 새로운 최고 검증 정확도: {best_val_acc*100:.2f}%")
    
    # Early Stopping 체크
    early_stopping(val_loss, model)
    
    if early_stopping.early_stop:
        print(f"\n{'='*60}")
        print(f"🛑 Early Stopping at Epoch {epoch + 1}")
        print(f"{'='*60}")
        break

# 최고 성능 모델 로드
print(f"\n{'='*60}")
print("최고 성능 모델 로드")
print(f"{'='*60}")
early_stopping.load_best_model(model)
print(f"✅ Epoch {best_epoch}의 모델 로드 완료 (Val Acc: {best_val_acc*100:.2f}%)")

print(f"\n{'='*60}")
print("학습 완료!")
print(f"{'='*60}")
print(f"총 학습 Epoch: {epoch + 1}/{EPOCHS}")
print(f"최고 검증 정확도: {best_val_acc*100:.2f}% (Epoch {best_epoch})")
print(f"최종 Val Loss: {early_stopping.best_loss:.4f}")

## 🔟 학습 결과 시각화

In [ ]:
# ==========================================
# 학습 결과 시각화 (Learning Rate 추가!)
# ==========================================

fig, axes = plt.subplots(1, 3, figsize=(18, 5))

# Loss 그래프
axes[0].plot(history['train_loss'], 'o-', label='Train Loss', color='blue')
axes[0].plot(history['val_loss'], 's-', label='Val Loss', color='orange')
axes[0].axvline(x=best_epoch-1, color='red', linestyle='--', alpha=0.5,
                label=f'Best (Epoch {best_epoch})')
axes[0].set_title('Training and Validation Loss', fontsize=14, fontweight='bold')
axes[0].set_xlabel('Epoch')
axes[0].set_ylabel('Loss')
axes[0].legend()
axes[0].grid(True, alpha=0.3)

# Accuracy 그래프
axes[1].plot([acc*100 for acc in history['train_acc']], 'o-',
             label='Train Acc', color='blue')
axes[1].plot([acc*100 for acc in history['val_acc']], 's-',
             label='Val Acc', color='orange')
axes[1].axvline(x=best_epoch-1, color='red', linestyle='--', alpha=0.5,
                label=f'Best (Epoch {best_epoch})')
axes[1].set_title('Training and Validation Accuracy', fontsize=14, fontweight='bold')
axes[1].set_xlabel('Epoch')
axes[1].set_ylabel('Accuracy (%)')
axes[1].legend()
axes[1].grid(True, alpha=0.3)

# Learning Rate 변화 (새로 추가!)
axes[2].plot(history['learning_rates'], 'o-', color='green')
axes[2].set_title('Learning Rate Schedule', fontsize=14, fontweight='bold')
axes[2].set_xlabel('Epoch')
axes[2].set_ylabel('Learning Rate')
axes[2].set_yscale('log')  # log scale로 보기
axes[2].grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

# 수치 요약
print("\n" + "="*60)
print("학습 요약")
print("="*60)
print(f"최고 Train Acc: {max(history['train_acc'])*100:.2f}%")
print(f"최고 Val Acc: {max(history['val_acc'])*100:.2f}% (Epoch {best_epoch})")
print(f"최종 Train Loss: {history['train_loss'][-1]:.4f}")
print(f"최종 Val Loss: {history['val_loss'][-1]:.4f}")
print(f"Train/Val Acc 차이: {(max(history['train_acc']) - max(history['val_acc']))*100:.2f}%p")
print("="*60)

## 1️⃣1️⃣ 테스트 평가

In [ ]:
# 테스트 데이터 평가
print("\n" + "="*60)
print("테스트 데이터 평가")
print("="*60)

test_loss, test_acc = eval_model(model, test_loader, criterion, device)

print(f"\n📊 Test Loss: {test_loss:.4f}")
print(f"📊 Test Accuracy: {test_acc*100:.2f}%")

In [ ]:
# 상세 평가 (Confusion Matrix & Classification Report)
def get_predictions(model, data_loader, device):
    """예측 결과 수집"""
    model.eval()
    all_preds = []
    all_labels = []
    
    with torch.no_grad():
        for batch in tqdm(data_loader, desc='Predicting'):
            input_ids = batch['input_ids'].to(device)
            attention_mask = batch['attention_mask'].to(device)
            labels = batch['label'].to(device)
            
            outputs = model(input_ids, attention_mask)
            _, predicted = torch.max(outputs, 1)
            
            all_preds.extend(predicted.cpu().numpy())
            all_labels.extend(labels.cpu().numpy())
    
    return np.array(all_preds), np.array(all_labels)

# 예측 수행
test_preds, test_true = get_predictions(model, test_loader, device)

# Classification Report
print("\n" + "="*60)
print("Classification Report")
print("="*60)
target_names = ['옹호', '중립', '비판']
print(classification_report(test_true, test_preds, target_names=target_names))

# Confusion Matrix
cm = confusion_matrix(test_true, test_preds)
plt.figure(figsize=(8, 6))
sns.heatmap(cm, annot=True, fmt='d', cmap='Blues',
            xticklabels=target_names,
            yticklabels=target_names)
plt.title('Confusion Matrix', fontsize=14, fontweight='bold')
plt.ylabel('True Label')
plt.xlabel('Predicted Label')
plt.tight_layout()
plt.show()

## 1️⃣2️⃣ 모델 저장

In [ ]:
# 모델 저장
MODEL_SAVE_PATH = 'stance_classifier_improved.pt'

torch.save({
    'model_state_dict': model.state_dict(),
    'optimizer_state_dict': optimizer.state_dict(),
    'best_val_acc': best_val_acc,
    'best_epoch': best_epoch,
    'history': history,
    'hyperparameters': {
        'MAX_LENGTH': MAX_LENGTH,
        'DROPOUT': DROPOUT,
        'BATCH_SIZE': BATCH_SIZE,
        'LEARNING_RATE': LEARNING_RATE,
        'WEIGHT_DECAY': WEIGHT_DECAY,
        'MAX_GRAD_NORM': MAX_GRAD_NORM
    }
}, MODEL_SAVE_PATH)

print(f"✅ 모델 저장 완료: {MODEL_SAVE_PATH}")

# Colab 환경에서 다운로드
if IN_COLAB:
    from google.colab import files
    files.download(MODEL_SAVE_PATH)
    print("📥 모델 다운로드 시작")

## 1️⃣3️⃣ 모델 로드 및 테스트

In [ ]:
# 저장된 모델 로드 (예시)
def load_model(model_path, device):
    """저장된 모델 로드"""
    checkpoint = torch.load(model_path, map_location=device)
    
    # 모델 초기화
    loaded_model = StanceClassifier(
        model_name=MODEL_NAME,
        num_classes=3,
        dropout=checkpoint['hyperparameters']['DROPOUT']
    ).to(device)
    
    # 가중치 로드
    loaded_model.load_state_dict(checkpoint['model_state_dict'])
    
    print(f"✅ 모델 로드 완료")
    print(f"   - Best Val Acc: {checkpoint['best_val_acc']*100:.2f}%")
    print(f"   - Best Epoch: {checkpoint['best_epoch']}")
    
    return loaded_model

# 사용 예시 (주석 해제하여 사용)
# loaded_model = load_model(MODEL_SAVE_PATH, device)
# test_loss, test_acc = eval_model(loaded_model, test_loader, criterion, device)

In [ ]:
# 실시간 예측 함수
def predict_stance(text, model, tokenizer, device, max_length=128):
    """
    단일 텍스트의 스탠스 예측
    
    Args:
        text: 예측할 뉴스 텍스트
        model: 학습된 모델
        tokenizer: 토크나이저
        device: 디바이스
        max_length: 최대 시퀀스 길이
    
    Returns:
        predicted_label: 예측된 라벨 (0, 1, 2)
        predicted_class: 예측된 클래스명
        probabilities: 각 클래스별 확률
    """
    model.eval()
    
    # 토크나이징
    encoding = tokenizer(
        text,
        add_special_tokens=True,
        max_length=max_length,
        padding='max_length',
        truncation=True,
        return_tensors='pt'
    )
    
    input_ids = encoding['input_ids'].to(device)
    attention_mask = encoding['attention_mask'].to(device)
    
    # 예측
    with torch.no_grad():
        outputs = model(input_ids, attention_mask)
        probabilities = torch.softmax(outputs, dim=1)
        predicted_label = torch.argmax(probabilities, dim=1).item()
    
    label_names = ['옹호', '중립', '비판']
    predicted_class = label_names[predicted_label]
    
    return predicted_label, predicted_class, probabilities.cpu().numpy()[0]

# 사용 예시
test_texts_sample = [
    "정부의 새로운 정책은 경제 회복에 크게 기여할 것으로 보입니다.",
    "오늘 국회에서 예산안이 통과되었습니다.",
    "이번 정책은 국민의 기대에 미치지 못하고 있습니다."
]

print("\n" + "="*60)
print("실시간 예측 테스트")
print("="*60)

for text in test_texts_sample:
    pred_label, pred_class, probs = predict_stance(text, model, tokenizer, device, MAX_LENGTH)
    
    print(f"\n텍스트: {text}")
    print(f"예측: {pred_class} (라벨 {pred_label})")
    print(f"확률: 옹호 {probs[0]:.2%}, 중립 {probs[1]:.2%}, 비판 {probs[2]:.2%}")

---

## 🎯 개선사항 요약

### ✅ 적용된 개선사항

1. **Early Stopping** (3 epoch patience)
   - 검증 손실이 개선되지 않으면 자동 중단
   - 최고 성능 모델 자동 저장 및 복원

2. **정규화 강화**
   - Dropout: 0.3 → 0.4
   - Weight Decay: 0.01 추가 (L2 regularization)

3. **학습 안정화**
   - Learning Rate: 5e-5 → 2e-5
   - Gradient Clipping: 1.0 (exploding gradient 방지)
   - ReduceLROnPlateau 스케줄러

4. **향상된 시각화**
   - Learning Rate 변화 추적
   - 최고 성능 epoch 표시
   - 상세한 학습 통계

### 🚀 예상 효과

- **과적합 완화**: Train/Val accuracy 차이 감소
- **일반화 성능 향상**: 테스트 정확도 개선
- **학습 효율성**: 불필요한 epoch 방지

---